In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import os
import random
import time

# # Inside the /Multimodal-Deep-Regression/notebooks
import sys
sys.path.append("..")

from util.utilities import train, evaluate, get_device
from util.data_utilities import  process_data, get_train_and_val_loader

# import all the models from models module
from models import Swin_Transformer
# from util.video_swin_transformer import SwinTransformer3D

device = get_device()

c:\Users\ahmed\anaconda3\envs\vstp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No CUDA runtime is found, using CUDA_HOME='C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.7'


You are using device: cpu


In [2]:
process_data(
    input_type='video_pack_1000', 
    addition_parameters={'first_n_videos': 5}, 
    verbose=True,
    device=device,
    skip_frames=True,
    frames_to_skip=5,
    resize_tensors=True
)

Currently processing: 6805640750861257989.mp4


c:\Users\ahmed\anaconda3\envs\vstp\Lib\site-packages\torchvision\io\video_reader.py:245: UserWarning: Accurate seek is not implemented for pyav backend
  warnings.warn("Accurate seek is not implemented for pyav backend")


Downsampling video tensors for every 5 frame
Original tensor size: (3, 322, 1280, 720)
New tensor size: torch.Size([3, 65, 1280, 720])
6805640750861257989
229.7
Done processing: ../data/video_packs/video_pack_1000/6805640750861257989.mp4.
X Tensor (torch.Size([3, 65, 1280, 720])) saved under: ../data/x_tensors/video_pack_1000/6805640750861257989_x_tensor.pt.
Y Tensor (torch.Size([1, 1])) saved under: ../data/y_tensors/video_pack_1000/6805640750861257989_y_tensor.pt.
metadata:
{'video': {'fps': [30.0], 'duration': [10.733333333333333]}, 'audio': {'framerate': [44100.0], 'duration': [10.796009070294785]}}


Currently processing: 6807877755045154053.mp4
Downsampling video tensors for every 5 frame
Original tensor size: (3, 356, 1024, 576)
New tensor size: torch.Size([3, 72, 1024, 576])
6807877755045154053
759.9
Done processing: ../data/video_packs/video_pack_1000/6807877755045154053.mp4.
X Tensor (torch.Size([3, 72, 1024, 576])) saved under: ../data/x_tensors/video_pack_1000/6807877755045

In [3]:
train_loader, val_loader = get_train_and_val_loader(
    input_type='video_pack_1000',
    batch_size=1,
    verbose=True
)

torch.Size([3, 65, 1280, 720])
torch.Size([3, 90, 576, 1024])
4
1


In [4]:
model = Swin_Transformer.Swin_Transformer_model()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())#, lr=0.01)

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")

EPOCHS = 20

train_losses = []
val_losses = []
start_time = time.time()

for epoch in range(EPOCHS):
    train_loss, avg_train_loss = train(model, train_loader, criterion, optimizer, device, verbose=True)
    val_loss, avg_val_loss = evaluate(model, val_loader, criterion, device, verbose=True)
    
    # record the losses
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)

    # print every num times epoch only
    num = 1
    if ((epoch+1) % num == 0) or epoch == 0:
        if epoch == 0:
            time_took = (time.time() - start_time) / 60
            print(f'First epoch took {time_took:.1f} minutes.')
        print(f'Epoch {epoch+1}/{EPOCHS}, Train_Loss: {train_loss:.2f}, Avg: {avg_train_loss:.2f}; Val_Loss: {val_loss:.2f}, Avg: {avg_val_loss:.2f}')


Total parameters: 27,579,632


Forward pass on batch with size: torch.Size([1, 3, 65, 1280, 720])


c:\Users\ahmed\anaconda3\envs\vstp\Lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


SwinTransformer3D(
  (patch_embed): PatchEmbed3D(
    (proj): Conv3d(3, 96, kernel_size=(4, 4, 4), stride=(4, 4, 4))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): BasicLayer(
      (blocks): ModuleList(
        (0): SwinTransformerBlock3D(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention3D(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path): Identity()
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU(approximate='none')
            (fc2): Linear(in_features=384, out_features=96, bias=True)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x12011520 and 1x1)